In [ ]:
%load_ext lab_black

import h5py
import os
import numpy as np
from typing import Dict, List, Optional, Tuple

from dataclasses import dataclass
from tqdm.auto import tqdm
from scipy.signal import savgol_filter
from scipy.interpolate import interp2d
from functools import lru_cache
import lmfit as lm


import plotly.graph_objects as go
import plotly.colors as pc
import matplotlib.pyplot as plt

import sys

sys.path.append(r"C:\Users\atully\Code\GitHub\ARPES Code\arpes-code-python")
from arpes_functions import fitting_functions, tr_functions, polygons


colors = pc.qualitative.D3
angstrom = "\u212B"
degree = "\u00B0"
Theta = "\u0398"

In [ ]:
m_e = 0.511 * 10**6  # eV/c^2
w_f = 4.2
hbar = 6.582 * 10 ** (-16)  # eV*s
c = 3 * 10**18

In [ ]:
def kpar(hv, theta_motor, phi_motor=0):
    # theta_rad_right = (theta_motor + 15) * np.pi / 180
    # theta_rad_left = (theta_motor - 15) * np.pi / 180
    theta_rad_right = (theta_motor + 55) * np.pi / 180
    theta_rad_left = (theta_motor - 55) * np.pi / 180
    phi_rad = (phi_motor) * np.pi / 180

    right_side = (
        np.sqrt(2 * m_e)
        / (hbar * c)
        * np.sqrt(hv - w_f)
        * np.sin(theta_rad_right)
        * np.cos(phi_rad)
    )
    left_side = (
        np.sqrt(2 * m_e)
        / (hbar * c)
        * np.sqrt(hv - w_f)
        * np.sin(theta_rad_left)
        * np.cos(phi_rad)
    )
    return left_side, right_side

In [ ]:
def create_circle(
    min_max_tuple,
    legend_name,
    line_color="LightSeaGreen",
    fill_color="LightSeaGreen",
    opacity=1,
    center_dot=False,
):
    # Circle
    radius = (min_max_tuple[1] - min_max_tuple[0]) / 2
    center_x = min_max_tuple[0] + radius

    # Create circle
    circle_shape = dict(
        type="circle",
        xref="x",
        yref="y",
        x0=min_max_tuple[0],
        y0=-radius,
        x1=min_max_tuple[1],
        y1=radius,
        line_color=line_color,
        fillcolor=fill_color,
        opacity=opacity,
    )

    # Create legend name
    legend_trace = go.Scatter(
        x=[None],
        y=[None],
        mode="lines",
        marker=dict(size=10, color=fill_color),
        name=legend_name,
        opacity=opacity,
    )

    # Create center dot
    if center_dot:
        center_x = min_max_tuple[0] + radius
        center_dot_trace = go.Scatter(
            x=[center_x],
            y=[0],
            mode="markers",
            marker=dict(size=10, color="black"),
            showlegend=False,
        )
        return circle_shape, legend_trace, center_dot_trace
    else:
        return circle_shape, legend_trace

In [ ]:
# title = f"Accessible Region of k<sub>||</sub><br>({Theta} = +/-15 degrees)"
title = f"Accessible Region of k<sub>||</sub><br>({Theta} = +/-55 degrees)"
yaxis_title = f"k<sub>||</sub> ({angstrom}<sup>-1</sup>)"
xaxis_title = f"k<sub>||</sub> ({angstrom}<sup>-1</sup>)"

In [ ]:
## Create circles ##
# Note: Analyser extremes are theta = +/- 40, and phi = (-40, 10).

circle_1, name_1, center_dot1 = create_circle(
    kpar(6.2, 0),
    legend_name="6.2 eV",
    line_color="skyblue",
    fill_color="skyblue",
    opacity=0.55,
    center_dot=True,
)
# circle_2 = create_circle(kpar(22, 0), color="aqua")
circle_3, name_3, center_dot2 = create_circle(
    kpar(22.8, 0),
    legend_name="22.8 eV",
    line_color="midnightblue",
    fill_color="midnightblue",
    opacity=0.35,
    center_dot=True,
)


## Plot Fig
fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    gridlines=False,
    equiv_axes=True,
    height=500,
    width=565,
    dtick_y=1,
)

fig.add_shape(circle_3)
fig.add_shape(circle_1)
fig.add_trace(center_dot1)
# fig.add_trace(center_dot2)
fig.add_trace(name_1)
fig.add_trace(name_3)

## Add Au Hexagon
hexagon = polygons.gen_polygon(6, 1.45, rotation=30)
fig = polygons.plot_polygon(
    hexagon,
    color="gold",
    fig=fig,
    show=False,
    dash=True,
    dash_width=3,
)

fig.data[-1].name = "Au(111) BZ"

## Add C60 Hexagon
hexagon = polygons.gen_polygon(6, 0.42, rotation=30)
fig = polygons.plot_polygon(
    hexagon,
    color="grey",
    fig=fig,
    show=False,
    dash=True,
    dash_width=3,
)

fig.data[-1].name = "C60 BZ"

# fig.update_layout(xaxis_range=(-0.75, 0.75), yaxis_range=(-0.75, 0.75), showlegend=True)

fig.show()